In [ ]:
from tests.test_attention import _attention_and_lse, _make_attn_inputs
from cs336_systems.flashattention_autograd_function_pytorch import FlashAttentionAutogradFunctionPytorch

In [ ]:
import torch
import math
import triton
import triton.language as tl
from einops import einsum, rearrange

@triton.jit
def flash_fwd_kernel(
    Q_ptr, K_ptr, V_ptr, O_ptr, L_ptr,
    stride_qb, stride_qq, stride_qd,
    stride_kb, stride_kk, stride_kd,
    stride_vb, stride_vk, stride_vd,
    stride_ob, stride_oq, stride_od,
    stride_lb, stride_lq,
    N_QUERIES, N_KEYS,
    scale,
    D: tl.constexpr,
    Q_TILE_SIZE: tl.constexpr,
    K_TILE_SIZE: tl.constexpr,
):
    # program indices
    query_tile_index = tl.program_id(0)
    batch_index = tl.program_id(1)

    Q_block_ptr = tl.make_block_ptr(
        Q_ptr + batch_index * stride_qb,
        shape=(N_QUERIES, D),
        strides=(stride_qq, stride_qd),
        offsets=(query_tile_index * Q_TILE_SIZE, 0),
        block_shape=(Q_TILE_SIZE, D),
        order=(1, 0),
    )

    K_block_ptr = tl.make_block_ptr(
        K_ptr + batch_index * stride_kb,
        shape=(N_KEYS, D),
        strides=(stride_kk, stride_kd),
        offsets=(0, 0),
        block_shape=(K_TILE_SIZE, D),
        order=(1, 0),
    )

    V_block_ptr = tl.make_block_ptr(
        V_ptr + batch_index * stride_vb,
        shape=(N_KEYS, D),
        strides=(stride_vk, stride_vd),
        offsets=(0, 0),
        block_shape=(K_TILE_SIZE, D),
        order=(1, 0),
    )

    O_block_ptr = tl.make_block_ptr(
        O_ptr + batch_index * stride_ob,
        shape=(N_QUERIES, D),
        strides=(stride_oq, stride_od),
        offsets=(query_tile_index * Q_TILE_SIZE, 0),
        block_shape=(Q_TILE_SIZE, D),
        order=(1, 0),
    )

    L_block_ptr = tl.make_block_ptr(
        L_ptr + batch_index * stride_lb,
        shape=(N_QUERIES, ),
        strides=(stride_lq,),
        offsets=(query_tile_index * Q_TILE_SIZE, ),
        block_shape=(Q_TILE_SIZE,),
        order=(0,),
    )

    O_block = tl.zeros((Q_TILE_SIZE, D), dtype=tl.float32)
    L_block = tl.zeros((Q_TILE_SIZE, ), dtype=tl.float32)
    m = tl.full((Q_TILE_SIZE,), float('-inf'), dtype=tl.float32)
    Q_block = tl.load(Q_block_ptr, boundary_check=(0,), padding_option='zero')

    for i in range(tl.cdiv(N_KEYS, K_TILE_SIZE)):
        # K_block = tl.load(K_block_ptr, boundary_check=(0,), padding_option='zero')
        # V_block = tl.load(V_block_ptr, boundary_check=(0,), padding_option='zero')
        K_block = tl.load(K_block_ptr)
        V_block = tl.load(V_block_ptr)

        S = scale * tl.dot(Q_block, tl.trans(K_block.to(tl.float32)))
        # print
        tl.device_print("S", S)

        m_curr = tl.maximum(m, tl.max(S, axis=-1))

        P = tl.exp(S - m_curr.expand_dims(axis=-1))

        alpha = tl.exp(m - m_curr)
        L_block = alpha * L_block  + tl.sum(P, axis=-1)
        # according to Claude, tl does not have `diag` so need to use broadcasting
        O_block = alpha[:, None] * O_block
        # using `acc` for `tl.float32`
        O_block = tl.dot(P.to(V_block.dtype), V_block, acc=O_block)
        m = m_curr

        # Move the pointer to next tile
        K_block_ptr = K_block_ptr.advance((K_TILE_SIZE, 0))
        V_block_ptr = V_block_ptr.advance((K_TILE_SIZE, 0))

    O_block = (1 / L_block)[:, None] * O_block
    L_block = m + tl.log(L_block)

    tl.store(O_block_ptr, O_block, boundary_check=(0,))
    tl.store(L_block_ptr, L_block, boundary_check=(0,))

class FlashAttentionAutogradFunctionTriton(torch.autograd.Function):
    @staticmethod
    def forward(ctx, Q, K, V, is_causal=False):
        # cache Q, K and V?
        batch_size, n_queries, D = Q.shape
        _, n_keys, _ = K.shape

        # reshape input tensor to 2D, i.e., remove batch dim
        Q_input_shape = Q.shape
        Q = rearrange(Q, "... d -> (...) d")
        K_input_shape = K.shape
        K = rearrange(K, "... d -> (...) d")
        V = rearrange(V, "... d -> (...) d")

        ctx.save_for_backward(Q, K, V)

        for t in [Q, K, V]:
            assert t.is_cuda, "Expected CUDA tensors"
            assert t.is_contiguous(), "Our pointer arithmetic will assume contiguous inputs"

        ctx.Q_TILE_SIZE = 16
        ctx.K_TILE_SIZE = 16
        ctx.Q_input_shape = Q_input_shape
        ctx.K_input_shape = K_input_shape

        O = torch.empty(Q.shape, device=Q.device)
        L = torch.zeros(Q.shape[0], device=Q.device)

        stride_qb = n_queries * D
        stride_qq = D
        stride_qd = 1
        stride_kb = n_keys * D
        stride_kk = D
        stride_kd = 1
        stride_vb = n_keys * D
        stride_vk = D
        stride_vd = 1
        stride_ob = stride_qb
        stride_oq = stride_qq
        stride_od = 1
        stride_lb = Q_input_shape[0]
        stride_lq = 1
        scale = 1 / (D ** 0.5)
         
        # print(stride_qb, stride_qq, stride_qd)
        # print(stride_kb, stride_kk, stride_kd)
        # print(stride_vb, stride_vk, stride_vd)
        # print(stride_ob, stride_oq, stride_od)
        # print(N_QUERIES, N_KEYS)
        flash_fwd_kernel[(math.ceil(O.shape[1]/ctx.Q_TILE_SIZE), Q_input_shape[0])](
            Q, K, V, O, L,
            stride_qb, stride_qq, stride_qd,
            stride_kb, stride_kk, stride_kd,
            stride_vb, stride_vk, stride_vd,
            stride_ob, stride_oq, stride_od,
            stride_lb, stride_lq,
            n_queries, n_keys,
            scale, D,
            ctx.Q_TILE_SIZE,
            ctx.K_TILE_SIZE,
        )

        return O.view(Q_input_shape)
    def backward(ctx):
        raise NotImplementedError
    
        

In [ ]:
device="cuda"
is_causal = False
impl = FlashAttentionAutogradFunctionTriton.apply

def _make_attn_inputs(device=None):
    torch.random.manual_seed(0)
    batch_size = 4
    n_queries = 128
    n_keys = 128
    D = 64
    q = torch.randn(batch_size, n_queries, D, device=device, requires_grad=True)
    k = torch.randn(batch_size, n_keys, D, device=device, requires_grad=True)
    v = torch.randn(batch_size, n_keys, D, device=device, requires_grad=True)
    do = torch.randn(batch_size, n_queries, D, device=device)

    return q, k, v, do

q, k, v, do = _make_attn_inputs(device)
o_ref, l_ref = _attention_and_lse(q, k, v, is_causal)
o = impl(q, k, v, is_causal)
# Q, K, V, _do = _make_attn_inputs(device)


In [ ]:
o

In [ ]:
o_ref

In [ ]:
def forward(Q, K, V, is_causal=False):
    device = Q.device
    batch_size, N_q, d = Q.shape
    _, N_k, _ = K.shape
    B_q = 16
    B_k = 16
    O = torch.zeros_like(Q, device=device)
    L = torch.zeros(batch_size, N_q, device=device)
    for i in range(0, N_q, B_q):
        Q_i = Q[:, i:i+B_q, :]
        O_i = torch.zeros_like(Q_i, device=device)
        l_i = torch.zeros(batch_size, B_q, device=device)
        m_i = torch.full((batch_size, B_q), float('-inf'), device=device)
        for j in range(0, N_k, B_k):
            K_j = K[:, j:j+B_k, :]
            V_j = V[:, j:j+B_k, :]
            S_j = 1 / d**(0.5) * einsum(Q_i, K_j, '... B_q d, ... B_k d -> ... B_q B_k')
            assert S_j.shape == (batch_size, B_q, B_k)
            print(S_j)
            print(S_j.dtype)

            m_curr = torch.max(torch.cat([m_i[:, :, None], S_j], axis=-1), axis=-1).values
            P_i = torch.exp(S_j - m_curr[:, :, None])
            assert P_i.shape == (batch_size, B_q, B_k)


            l_i = torch.exp(m_i - m_curr)*l_i + torch.sum(P_i, axis=-1)
            
            _ = torch.diag_embed(torch.exp(m_i - m_curr))
            O_i = einsum(_, O_i, '... B_q B_q, ... B_q d -> ... B_q d') + einsum(P_i, V_j, '... B_q B_k, ... B_k d -> ... B_q d')
            assert O_i.shape == (batch_size, B_q, d)

            m_i = m_curr
        O_i = einsum(torch.diag_embed(1 / l_i), O_i, '... B_q B_q, ... B_q d -> ... B_q d')
        L_i = m_i + torch.log(l_i)

        O[:, i:i+B_q] += O_i
        L[:, i:i+B_q] += L_i

forward(q, k, v)

In [ ]:
o_ref